In [7]:
from sutime import SUTime
import pandas as pd
import datetime
from dateutil.parser import *
import re

In [133]:
validation_data = pd.DataFrame(columns=['text', 'expected_datetime'])
current_date = datetime.datetime.now()
current_date = current_date.replace(microsecond=0)
validation_data = validation_data.append([
    {'text': 'I will be there tomorrow', 'expected_datetime': (current_date + datetime.timedelta(days=1)).date()},
    {'text': 'I will be there today', 'expected_datetime': current_date.date()},
    {'text': 'I will be there yesterday', 'expected_datetime': (current_date - datetime.timedelta(days=1)).date()},
    {'text': 'I will be there next week', 'expected_datetime': (current_date + datetime.timedelta(days=7)).date()},
    {'text': 'I will be there last week', 'expected_datetime': (current_date - datetime.timedelta(days=7)).date()},
    {'text': 'I will be there next month', 'expected_datetime': (current_date + datetime.timedelta(weeks=4)).date().replace(day=1)},
    {'text': 'I will be there last month', 'expected_datetime': (current_date - datetime.timedelta(weeks=4)).date().replace(day=1)},
    {'text': 'I will be there next year', 'expected_datetime': (current_date + datetime.timedelta(weeks=52)).date().replace(day=1, month=1)},
    {'text': 'I will be there last year', 'expected_datetime': (current_date - datetime.timedelta(weeks=52)).date().replace(day=1, month=1)},
    {'text': 'I will be there the day after tomorrow', 'expected_datetime': (current_date + datetime.timedelta(days=2)).date()},
    {'text': 'I will be there the day before yesterday', 'expected_datetime': (current_date - datetime.timedelta(days=2)).date()},
    {'text': 'I will be there the day before today', 'expected_datetime': (current_date - datetime.timedelta(days=1)).date()},
    {'text': 'I will be there the day after today', 'expected_datetime': (current_date + datetime.timedelta(days=1)).date()},
    {'text': 'I will be there the week after the next', 'expected_datetime': (current_date + datetime.timedelta(days=14)).date()},
    {'text': 'I will be there the week before the last', 'expected_datetime':  (current_date - datetime.timedelta(days=14)).date()},
    {'text': 'I will be there the month after the next', 'expected_datetime': (current_date + datetime.timedelta(weeks=8)).date().replace(day=1)},
    {'text': 'I will be there the month before the last', 'expected_datetime': (current_date - datetime.timedelta(weeks=8)).date().replace(day=1)},
    {'text': 'I will be there the year after the next', 'expected_datetime': (current_date + datetime.timedelta(weeks=104)).date().replace(day=1, month=1)},
    {'text': 'I will be there the year before the last', 'expected_datetime': (current_date - datetime.timedelta(weeks=104)).date().replace(day=1, month=1)},
    {'text': 'I will be there at 10:00', 'expected_datetime': current_date.replace(hour=10, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there at ten o\'clock am', 'expected_datetime': current_date.replace(hour=10, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there in 10 minutes', 'expected_datetime': current_date + datetime.timedelta(minutes=10)},
    {'text': 'I will be there in 10 hours', 'expected_datetime': current_date + datetime.timedelta(hours=10)},
    {'text': 'I will be there in ten minutes', 'expected_datetime': current_date + datetime.timedelta(minutes=10)},
    {'text': 'I will be there in ten hours', 'expected_datetime': current_date + datetime.timedelta(hours=10)},
    {'text': 'I will be there in 10 minutes and 10 hours', 'expected_datetime': current_date + datetime.timedelta(hours=10, minutes=10)},
    {'text': 'I was there 10 minutes ago', 'expected_datetime': current_date - datetime.timedelta(minutes=10)},
    {'text': 'I was there 10 hours ago', 'expected_datetime': current_date - datetime.timedelta(hours=10)},
    {'text': 'I will be there this morning', 'expected_datetime': current_date.replace(hour=8, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there this afternoon', 'expected_datetime': current_date.replace(hour=12, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there this evening', 'expected_datetime': current_date.replace(hour=18, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there this night', 'expected_datetime': current_date.replace(hour=22, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there this coming Friday', 'expected_datetime': (current_date + (datetime.timedelta(days=(4-current_date.weekday()) % 7))).replace(hour=0, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there next Friday', 'expected_datetime': (current_date + (datetime.timedelta(days=(4-current_date.weekday()) % 7))).replace(hour=0, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there next week Friday', 'expected_datetime': (current_date + (datetime.timedelta(days=(4-current_date.weekday()) % 7, weeks=1))).replace(hour=0, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there last Friday', 'expected_datetime': (current_date - (datetime.timedelta(days=(current_date.weekday()-4) % 7))).replace(hour=0, minute=0, second=0, microsecond=0)},
    {'text': 'I will be there next Monday at ten o\'clock am', 'expected_datetime': (current_date + (datetime.timedelta(days=(7-current_date.weekday()) % 7))).replace(hour=10, minute=0, second=0, microsecond=0)},
])

C:\Users\elibr\AppData\Local\Temp\ipykernel_15256\629715250.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_data = validation_data.append([


In [131]:
sutime = SUTime()

In [135]:
final_data = pd.DataFrame(columns=['text', 'expected_datetime', 'result_parsed_datetime', 'result_string'])
for index, row in validation_data.iterrows():
    result = sutime.parse(row['text'])
    
    date = result[0]['value']
    # switch of formats (yyyy-mm-dd, yyyy-Mmm)
    if re.match(r'^\d{4}-\d{2}-\d{2}$', date): # yyyy-mm-dd
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%d')
    elif re.match(r'^\d{4}-W\d{2}$', date): # yyyy-Www
        parsed_date = datetime.datetime.strptime(date + '-1', "%Y-W%W-%w")
    elif re.match(r'^\d{4}-\d{2}$', date): # yyyy-mm
        parsed_date = datetime.datetime.strptime(date + '-01', "%Y-%m-%d")
    elif re.match(r'^\d{4}$', date): # yyyy 
        parsed_date = datetime.datetime.strptime(date + '-01-01', "%Y-%m-%d")
    elif re.match(r'^\d{2}:\d{2}$', date): # hh:mm
        parsed_date = datetime.datetime.strptime(date, '%H:%M')
    elif re.match(r'^\d{4}-\d{2}-\d{2}T\d{2}:\d{2}$', date): # yyyy-mm-ddThh:mm
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M')
    elif re.match(r'^PT\d{2}H\d{2}M$', date): # PTxxHxxM
        parsed_date = current_date + datetime.timedelta(hours=int(date[2:4]), minutes=int(date[5:7]))
    elif re.match(r'^PT\d{2}H$', date): # PTxxH
        parsed_date = current_date + datetime.timedelta(hours=int(date[2:4]))
    elif re.match(r'^PT\d{2}M$', date): # PTxxM
        parsed_date =  current_date + datetime.timedelta(minutes=int(date[2:4]))
    elif re.match(r'^\d{4}-\d{2}-\d{2}TMO$', date): # yyyy-mm-ddTMO
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%dTMO').replace(hour=8, minute=0, second=0, microsecond=0)
    elif re.match(r'^\d{4}-\d{2}-\d{2}TAF$', date): # yyyy-mm-ddTAF
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%dTAF').replace(hour=12, minute=0, second=0, microsecond=0)
    elif re.match(r'^\d{4}-\d{2}-\d{2}TEV$', date): # yyyy-mm-ddTEV
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%dTEV').replace(hour=18, minute=0, second=0, microsecond=0)
    elif re.match(r'^\d{4}-\d{2}-\d{2}TNI$', date): # yyyy-mm-ddTNI
        parsed_date = datetime.datetime.strptime(date, '%Y-%m-%dTNI').replace(hour=22, minute=0, second=0, microsecond=0)
    else:
        raise Exception(f'Unknown date format: {result} for text: {row["text"]}')
    
    final_data = pd.concat([final_data, pd.DataFrame([{
        'text': row['text'],
        'expected_datetime': row['expected_datetime'],
        'result_parsed_datetime': parsed_date,
        'result_string': result[0]['text'],
        'result_value': result[0]['value'],
    }])])
    

In [137]:
not_matching = final_data[final_data['expected_datetime'] != final_data['result_parsed_datetime']]
print(f'Not matching: {len(not_matching)}')
accuracy = 1 - len(not_matching) / len(final_data)
print(f'Accuracy: {accuracy}')


Not matching: 12
Accuracy: 0.6756756756756757
